In [1]:
with open("./input.txt", "r") as file: 
    data = [
        [int(x) for x in row] for row 
        in file.read().strip().split("\n")
    ]

# Part 1

In [2]:
import pandas as pd 
import functools 

def solve(data): 
    data = pd.DataFrame(data) 
    
    # cumulative maximum heights in each directions
    # right (r), left (l), top (t) and bottom (b)
    # 
    # we shift by 1 in each direction + fill the first item with -1
    # because the outer-most tree is always visible from the outside
    cumheights = {
        "l": data.shift(1, axis=1).fillna(-1).cummax(axis=1), 
        "r": data.iloc[:,::-1].cummax(axis=1).shift(1, axis=1).fillna(-1).iloc[:,::-1],
        "t": data.shift(1,axis=0).fillna(-1).cummax(axis=0),
        "b": data.iloc[::-1].shift(1,axis=0).fillna(-1).cummax(axis=0).iloc[::-1]
    }
    
    # return map of points which higher (gt, strict) than the cumulative
    # heights in each direction
    mapping = functools.reduce(
        lambda acc, cumheight: acc | data.gt(cumheight), 
        cumheights.values(), 
        data.applymap(lambda x: False)
    ) 
    
    return mapping.sum().sum()
    
solve(data)

1803

# Part 2

In [3]:
def solve(data):
    # number of rows, number of columns
    rows, columns = (len(data), len(data[1]))

    # array
    scores = [[0 for _ in range(columns)] for _ in range(rows)]

    for r in range(rows):
        for c in range(columns):
            distances = {}

            # count number of trees to the left that
            # are lower than the current tree
            for x in range(c - 1, -1, -1):
                if data[r][x] >= data[r][c]:
                    distances.update({"left": c - x})
                    break
            else:
                distances.update({"left": c})

            # compute number of trees to the right
            for x in range(c + 1, columns):
                if data[r][x] >= data[r][c]:
                    distances.update({"right": x - c})
                    break
            else:
                distances.update({"right": columns - c - 1})

            # compute number of tree above
            for y in range(r - 1, -1, -1):
                if data[y][c] >= data[r][c]:
                    distances.update({"above": r - y})
                    break
            else:
                distances.update({"above": r})

            # compute number of trees below
            for y in range(r + 1, rows):
                if data[y][c] >= data[r][c]:
                    distances.update({"below": y - r})
                    break
            else:
                distances.update({"below": rows - r - 1})

            scores[r][c] = functools.reduce(
                lambda acc, curr: acc * curr, 
                distances.values(), 
                1
            )
    
    # return the highest scenic score
    maximum = pd.DataFrame(scores).max().max()

    return maximum

solve(data)

268912